# Języki Programowania Python i R


## dr inż. Patryk Jasik
### Division of Theoretical Physics and Quantum Information
### Institute of Physics and Computer Science
### Faculty of Applied Physics and Mathematics
### Gdansk University of Technology

In [ ]:
#ładowanie bibliotek
from pandas import read_csv
from pandas import datetime
import random as rnd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metr

In [ ]:
%config Completer.use_jedi = False

In [ ]:
#ustawienia wykresów i wyświetlanie liczb zmiennoprzecinkowych
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=8, suppress=True)

In [ ]:
#Modelowanie szeregów czasowych
#klasyczny podział na zbiór treningowy i testowy oraz walidacja krzyżowa nie będę tutaj skuteczne


#Podział szeregu na zbiór treningowy i testowy - backtesting lub hindcasting
#train-test split - z zachowaniem chronologii szeregu

series = read_csv('dane/sunspots.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
X = series.values
train_size = int(len(X) * 0.66)
train, test = X[0:train_size], X[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

In [ ]:
#i jeszcze wykres zbioru treningowego i testowego
plt.plot(train)
plt.plot([None for i in train] + [x for x in test])
plt.show()

In [ ]:
#multi train-test split - podział szeregu czasowego na kilka zbiorów
#coś w rodzaju walidacji krzyżowej - metoda KFold

from sklearn.model_selection import TimeSeriesSplit

series = read_csv('dane/sunspots.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
X = series.values
splits = TimeSeriesSplit(n_splits=4)
plt.figure(1)
index = 1
for train_index, test_index in splits.split(X):
    train = X[train_index]
    test = X[test_index]
    print('Observations: %d' % (len(train) + len(test)))
    print('Training Observations: %d' % (len(train)))
    print('Testing Observations: %d' % (len(test)))
    plt.subplot(410 + index)
    plt.plot(train)
    plt.plot([None for i in train] + [x for x in test])
    index += 1
plt.show()

In [ ]:
#konstrukcja metody postępowania w przewidywaniu wartości szeregu czasowego
#budowa modelu referencyjnego i określenie wartości referencyjnej (bazowej) błędu

#1. wczytanie zbioru
def parser(x):
    return datetime.strptime('202'+x, '%Y-%m')

series = read_csv('dane/shampoo-sales.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True, date_parser=parser)

In [ ]:
series.plot()

In [ ]:
#2. tworzymy zmienną, która jest naszym szeregiem przesuniętym o jeden krok czasowy
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']
print(dataframe.head(5))

In [ ]:
#3. dzielimy szereg na część uczącą i testową
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]

train_X, train_y = train[:,0], train[:,1]

test_X, test_y = test[:,0], test[:,1]

In [ ]:
train_X

In [ ]:
train_y

In [ ]:
#4. tworzymy funkcję zwracającą wartość predykcji modelu naiwnego
def model_persistence(x):
    return x

In [ ]:
#5. testujemy model za pomocą metody walidacji postępującej
predictions = list()
for x in test_X:
    yhat = model_persistence(x)
    predictions.append(yhat)

rmse = np.sqrt(metr.mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
#6. wyświtalmy wyniki predykcji i dane oryginalne
plt.plot(train_y)
plt.plot([None for i in train_y] + [x for x in test_y])
plt.plot([None for i in train_y] + [x for x in predictions])
plt.show()

In [ ]:
#ocena jakości modeli predykcyjnych poprzez wizualizację

#budujemy naiwny model predykcyjny
series = read_csv('dane/daily-total-female-births.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

# tworzymy nową zmienną 
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# dzielimy szereg na zbiór treningowy i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

In [ ]:
# predykcja
predictions = [x for x in test_X]

# wyznaczamy błędy (residuals)
residuals = [test_y[i]-predictions[i] for i in range(len(predictions))]
residuals = pd.DataFrame(residuals)
print(residuals.head(10))

In [ ]:
# liniowy wykres błędów
residuals.plot()
plt.show()

In [ ]:
# statystyki błędów predykcji
print(residuals.describe())

In [ ]:
#histogram błędów i ich rozkład
residuals.hist()
plt.show()

In [ ]:
residuals.plot(kind='kde')
plt.show()

In [ ]:
#wykres Q-Q albo wykres kwantylowy
#możemy sprawdzić w ten sposób nomralność rozkładu błędów

from statsmodels.graphics.gofplots import qqplot

series = read_csv('dane/daily-total-female-births.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

predictions = [x for x in test_X]

residuals = [test_y[i]-predictions[i] for i in range(len(predictions))]
residuals = np.array(residuals)

qqplot(residuals, line='r')
plt.show()

In [ ]:
#wykres autokorelacji

from pandas.plotting import autocorrelation_plot

autocorrelation_plot(residuals)
plt.show()

In [ ]:
#inżynieria cech dla szeregów czasowych

#zagadnienie regresyjne
#wczytujemy dane
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

#tworzymy nową cechę poprzez przesunięcie danych w czasie
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

#budujemy cechę jako najbliższą mniejszą liczbę podzielną przez 5 
for i in range(len(dataframe['t+1'])):
    dataframe['t+1'][i] = int(dataframe['t+1'][i] / 5) * 5.0

print(dataframe.head(5))

In [ ]:
#budowanie cech dla klasyfikacji

series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                  index_col=0, parse_dates=True, squeeze=True)

values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

#dyskretyzujemy zmienną ciągłą
for i in range(len(dataframe['t+1'])):
    value = dataframe['t+1'][i]
    if value < 10.0:
        dataframe['t+1'][i] = 0
    elif value >= 20.0:
        dataframe['t+1'][i] = 2
    else:
        dataframe['t+1'][i] = 1

dataframe['t+1'].value_counts()

In [ ]:
#budowanie cech dla predykcji dłuższych horyzontów czasowych

series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

# tworzymy zbiór przesuniętych w czasie cech
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values, values.shift(-1),
                       values.shift(-2), values.shift(-3),
                       values.shift(-4), values.shift(-5),
                       values.shift(-6)], axis=1)
dataframe.columns = ['t', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7']



In [ ]:
dataframe.head(20)

### https://tsfresh.readthedocs.io/en/latest/index.html
### https://tsfel.readthedocs.io/en/latest/index.html

In [ ]:
import datetime
aaa = datetime.date(2022, 4, 3).isocalendar()
aaa

In [ ]:
#MODELOWANIE

#Model autoregresyjny -> X(t+1) = b0 + b1*X(t) + b2*X(t-1)


#przed przystąpieniem do modelowania sprawdzamy czy w szeregu występują korelacje

from pandas.plotting import lag_plot
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)
lag_plot(series)
plt.show()

In [ ]:
#sprawdzamy wielkość korelacji i tworzymy wykres autokorelacji (dwie wersje)
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf

series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']
result = dataframe.corr()
print(result)


In [ ]:
autocorrelation_plot(series)
plt.xlim(0,30)
plt.show()


In [ ]:
plot_acf(series, lags=30)
plt.show()

In [ ]:
#budujemy naiwny model bazowy

from sklearn.metrics import mean_squared_error
from math import sqrt

#wczytywanie danych
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

#tworzymy zmienną przesuniętą w czasie
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

#tworzymy zbiór treningowy i testowy
X = dataframe.values
train, test = X[1:len(X)-7], X[len(X)-7:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model bazowy
def model_persistence(x):
    return x

#walidacja krocząca
predictions = list()
for x in test_X:
    yhat = model_persistence(x)
    predictions.append(yhat)

#obliczamy błąd predykcji
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

#wizualiazacja predykcji
plt.plot(test_y)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#Model AR

#wczytujemy odpowiedni pakiet
from statsmodels.tsa.ar_model import AR

#wczytywanie danych
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

#tworzymy zbiór treningowy i testowy
X = series.values
train, test = X[1:len(X)-7], X[len(X)-7:]

#budujemy model autoregresyjny
model = AR(train)
model_fit = model.fit()
print('Lag: %s' % model_fit.k_ar)
print('Coefficients: %s' % model_fit.params)
print(len(model_fit.params))

#predykcja
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
for i in range(len(predictions)):
    print('predicted=%f, expected=%f' % (predictions[i], test[i]))
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# wykres predykcji
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#budowanie modelu AR dla nowych wartości
#korzystamy ze współczynników bazowego modelu AR
#yhat = b0 + b1*X1 + b2*X2 + ... + bn*Xn

#wczytywanie danych
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

#tworzymy zbiór treningowy i testowy
X = series.values
train, test = X[1:len(X)-7], X[len(X)-7:]

#budujemy model autoregresyjny
model = AR(train)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params


#robimy predykcję kolejnych punktów ze zbioru testowego,
#korzystając ze współczynników wyznaczonych dla zbioru treningowego
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test)):
    length = len(history)
    lag = [history[i] for i in range(length-window,length)]
    yhat = coef[0]
    for d in range(window):
        yhat += coef[d+1] * lag[window-d-1]
    obs = test[t]
    predictions.append(yhat)
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

#wykres predykcji
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#tworzymy model naiwny

# wczytywanie danych
series = pd.read_csv('dane/daily-total-female-births.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
predictions = [x for x in test_X]

# sprawdzamy jakość modelu naiwnego
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

# wyznaczamy błędy (reszty)
residuals = [test_y[i]-predictions[i] for i in range(len(predictions))]
residuals = pd.DataFrame(residuals)
print(residuals.head())

In [ ]:
#mamy teraz szereg reszt, który możemy modelować za pomocą modeli AR

#error(t+1) = b0 + b1*error(t) + b2*error(t-1) + ... + bn*error(t-n)

#wczytywanie danych
series = pd.read_csv('dane/daily-total-female-births.csv', header=0,
                     index_col=0, parse_dates=True, squeeze=True)

# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
train_pred = [x for x in train_X]

# obliczamy reszty
train_resid = [train_y[i]-train_pred[i] for i in range(len(train_pred))]

# trenujemy model AR dla reszt
model = AR(train_resid)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params
print('Lag=%d, Coef=%s' % (window, coef))



In [ ]:
#wykorzystamy wyniki do predykcji błędu (reszt)

# wykorzystujemy walidację kroczącą dla zbioru testowego z wykorzystaniem okna
history = train_resid[len(train_resid)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
expected_error = list()
for t in range(len(test_y)):
    # model naiwyny
    yhat = test_X[t]
    error = test_y[t] - yhat
    expected_error.append(error)
    # predykcja błędu
    length = len(history)
    lag = [history[i] for i in range(length-window,length)]
    pred_error = coef[0]
    for d in range(window):
        pred_error += coef[d+1] * lag[window-d-1]
    predictions.append(pred_error)
    history.append(error)
    print('predicted error=%f, expected error=%f' % (pred_error, error))

# wykres reszt
plt.plot(expected_error)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#poprawiamy naiwną predykcję przy użyciu predykcji reszt
#improved_forecast = forecast + estimated_error

#wczytywanie danych
series = pd.read_csv('dane/daily-total-female-births.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
train_pred = [x for x in train_X]

# obliczamy reszty
train_resid = [train_y[i]-train_pred[i] for i in range(len(train_pred))]

# trenujemy model AR dla reszt
model = AR(train_resid)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params

# wykorzystujemy walidację kroczącą dla zbioru testowego z wykorzystaniem okna
history = train_resid[len(train_resid)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test_y)):
    # model naiwyny
    yhat = test_X[t]
    error = test_y[t] - yhat
    # predykcja reszt
    length = len(history)
    lag = [history[i] for i in range(length-window,length)]
    pred_error = coef[0]
    for d in range(window):
        pred_error += coef[d+1] * lag[window-d-1]
    # poprawiamy predykcję przy użyciu reszt
    yhat = yhat + pred_error
    predictions.append(yhat)
    history.append(error)
    print('predicted=%f, expected=%f' % (yhat, test_y[t]))

# błąd predykcji
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

# wykres predykcji
plt.plot(test_y)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
#ARIMA - AutoRegressive Integrated Moving Average
#AR I(różnicowanie) MA
#ARIMA(p,d,q)
#p - rzęd autoregresji (liczba wcześniejszych wartości)
#d - rząd różnicowania
#q - rząd średniej ruchomej


# wczytujemy dane
def parser(x):
    return pd.datetime.strptime('201'+x, '%Y-%m')

series = pd.read_csv('dane/shampoo-sales.csv', header=0, index_col=0,
                     parse_dates=True, squeeze=True, date_parser=parser)

print(series.head())

# wykresy
series.plot()
plt.show()




In [ ]:
#na podstawie wykresu autokorelacji możemy ustalić lag dla modelu ARIMA (jaki?)

from statsmodels.tsa.arima_model import ARIMA

# budujemy model i trenujemy
model = ARIMA(series, order=(6, 1, 1))
model_fit = model.fit()

# podsumowanie trenowania
print(model_fit.summary())


In [ ]:
# wykresy reszt
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()



In [ ]:
# rozkład reszt
residuals.plot(kind='kde')
plt.show()


In [ ]:

# statystyki reszt
print(residuals.describe())

In [ ]:
# zbiór uczący i testowy
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# walidacja krocząca
for t in range(len(test)):
    model = ARIMA(history, order=(6, 1, 1))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# błąd predykcji
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# wykres predykcji
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
predictions = [x for x in test_X]

# sprawdzamy jakość modelu naiwnego
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
#wczytywanie danych
series = pd.read_csv('dane/daily-total-female-births.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

# zbiór uczący i testowy
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# walidacja krocząca
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# błąd predykcji
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# wykres predykcji
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
predictions = [x for x in test_X]

# sprawdzamy jakość modelu naiwnego
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
#wczytywanie danych
series = pd.read_csv('dane/daily-minimum-temperatures.csv', header=0, index_col=0,
                  parse_dates=True, squeeze=True)

# zbiór uczący i testowy
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# walidacja krocząca
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# błąd predykcji
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# wykres predykcji
plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
# tworzenie cechy dla modelu naiwnego
values = pd.DataFrame(series.values)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']

# zbiór uczący i testowy
X = dataframe.values
train_size = int(len(X) * 0.66)
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# model naiwny
predictions = [x for x in test_X]

# sprawdzamy jakość modelu naiwnego
rmse = sqrt(mean_squared_error(test_y, predictions))
print('Test RMSE: %.3f' % rmse)

In [ ]:
#ładowanie bibliotek
from pandas import read_csv
import random as rnd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metr
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from datetime import datetime

import warnings

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.ar_model import ARResults #do wczytywania modeli

In [ ]:
#wczytujemy dane
series = read_csv('dane/daily-minimum-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

#długość szeregu czasowego
print(series.shape)

#funkcja ACF z zaznaczonym 95% przedziałem ufności
#wartości na zewnątrz tego obszaru są najprawdopodobniej prawdziwymi korelacjami,
#natomiast wewnątrz mogą być statystycznym szumem

#Wiemy, że ACF opisuje autokorelację między obserwacją a inną obserwacją we wcześniejszym kroku czasowym
#zawiera informację o bezpośredniej i pośredniej zależności
plot_acf(series,lags=3649)
plt.show()

In [ ]:
#funkcja ACF dla pierwszych 50 obserwacji (lagów)
plot_acf(series, lags=100)
plt.show()

In [ ]:
#funkcja PACF może być liczona maksymalnie dla połowy obserwacji, ale lepiej wybrać dużo mniej obserwacji
print(3650/2-1)

plot_pacf(series, lags=1250)
plt.show()

In [ ]:
#Wiemy, że PACF opisuje tylko bezpośredni związek między obserwacją a jej opóźnieniem
#zatem nie obserwuje się korelacji dla wartości opóźnień powyżej jakiegoś k
plot_pacf(series, lags=50)
plt.show()

In [ ]:
# funkcja określająca jakość modelu ARIMA za pomocą metryki RMSE dla zadanych parametrów (p,d,q)
def evaluate_arima_model(X, arima_order):
    # przygotowanie zbioru treningowego i testowego
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    
    # predykcja
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    
    # wyznaczenie błędu predykcji
    rmse = metr.mean_squared_error(test, predictions, squared=False)
    return rmse

In [ ]:
# ewaluacja modeli ARIMA z różnymi kombinacjami parametrów p, d, q 
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA %s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
# wczytanie danych
def parser(x):
    return datetime.strptime('202'+x, '%Y-%m')

series = read_csv('dane/shampoo-sales.csv', header=0, index_col=0, parse_dates=True, squeeze=True, date_parser=parser)


series.plot()
plt.show()

In [ ]:
#ACF i PACF
print(series.shape)

plot_acf(series,lags=12)
plt.show()

In [ ]:
plot_pacf(series,lags=12)
plt.show()

In [ ]:
# ewaluacja modeli (UWAGA - TRWA BARDZO DŁUGO - DO TESTÓW ZMNIEJSZAMY SIATKĘ)
p_values = [0,1,2,3,4,5,6,7,8]
d_values = range(0, 3)
q_values = range(0, 3)

warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

### https://facebook.github.io/prophet/
### https://unit8co.github.io/darts/#
### https://pycaret.readthedocs.io/en/latest/index.html